In [1]:
import os
import json
import pandas as pd

# Define the correct experiment directory
experiment_dir = "../outputs"  # Adjusted path to match Jupyter Notebook CWD
output_csv = "experiment_summary.csv"

# Initialize data storage
experiment_data = []


# Function to extract classification report
def parse_classification_report(file_path):
    metrics = {}
    with open(file_path, "r") as f:
        lines = f.readlines()
        for line in lines:
            parts = line.split()
            if len(parts) == 5 and parts[0].isdigit():  # Class-wise metrics
                class_name = parts[0]
                metrics[f"Precision_{class_name}"] = float(parts[1])
                metrics[f"Recall_{class_name}"] = float(parts[2])
                metrics[f"F1-score_{class_name}"] = float(parts[3])
            elif "accuracy" in line.lower():  # Overall accuracy
                metrics["Accuracy"] = float(parts[-2])
    return metrics


# Function to extract accuracy/loss from history CSV
def parse_history_csv(file_path):
    history_data = pd.read_csv(file_path)
    last_epoch = history_data.iloc[-1]  # Get last row (final epoch results)
    return {
        "Train Accuracy": last_epoch.get("accuracy", None),
        "Val Accuracy": last_epoch.get("val_accuracy", None),
        "Train Loss": last_epoch.get("loss", None),
        "Val Loss": last_epoch.get("val_loss", None),
    }


# Mapping for model-specific history file names
history_files = {
    "draft_mobilenetv2": "history_mobilenet.csv",
    "draft_sigmoid": "history_sigmoid.csv",
    "draft_softmax": "history_softmax.csv",
}

# Iterate over each model version
for model_version in ["draft_mobilenetv2", "draft_sigmoid", "draft_softmax"]:
    model_path = os.path.join(experiment_dir, model_version)
    if os.path.isdir(model_path):
        for trial in ["1-trial", "2-trial", "3-trial", "4-trial"]:
            trial_path = os.path.join(model_path, trial)

            # Initialize result storage
            trial_result = {
                "Model Version": model_version,
                "Trial #": trial,
                "Train Accuracy": None,
                "Val Accuracy": None,
                "Train Loss": None,
                "Val Loss": None,
            }

            # Load model-specific history CSV
            history_filename = history_files.get(model_version, "history.csv")
            history_path = os.path.join(trial_path, history_filename)
            if os.path.exists(history_path):
                trial_result.update(parse_history_csv(history_path))
            else:
                print(f"Missing history file: {history_path}")

            # Load classification report
            class_report_path = os.path.join(
                trial_path, "classification_report_test.txt"
            )
            if os.path.exists(class_report_path):
                trial_result.update(parse_classification_report(class_report_path))
            else:
                print(f"Missing classification report: {class_report_path}")

            # Append results
            experiment_data.append(trial_result)

# Convert results to DataFrame
df_experiments = pd.DataFrame(experiment_data)

# Save to CSV manually
df_experiments.to_csv(output_csv, index=False)
print(f"Summary saved as '{output_csv}'.")

Summary saved as 'experiment_summary.csv'.


In [2]:
import pandas as pd

df = pd.read_csv("experiment_summary.csv")
df.head()  

,Model Version,Trial #,Train Accuracy,Val Accuracy,Train Loss,Val Loss,Accuracy
0,draft_mobilenetv2,1-trial,1.000000,0.511905,0.014185,11.079432,0.757
1,draft_mobilenetv2,2-trial,1.000000,0.997619,0.098224,0.093501,1.000
2,draft_mobilenetv2,3-trial,1.000000,0.997619,0.231397,0.226450,0.996
3,draft_mobilenetv2,4-trial,0.996603,0.985714,0.354951,0.375562,0.992
4,draft_sigmoid,1-trial,0.990766,0.988095,0.028232,0.067577,0.999
